## Magentic-One
https://microsoft.github.io/autogen/dev/user-guide/agentchat-user-guide/magentic-one.html

In [ ]:
%pip install "autogen-ext[file-surfer]==0.4.0.dev13"

## Load Azure Configuration

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)


# Helper Method to see the responses per agent

In [7]:
def get_agent_responses(response):
    # ANSI escape code for bold text
    bold_start = "\033[1m"
    bold_end = "\033[0m"

    # ANSI escape code for red text
    red_start = "\033[31m"
    red_end = "\033[0m"

    print(f"{bold_start}****************Agent Responses****************{bold_end}")

    for messages in response.messages:
        source = messages.source
        print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
        print(messages.content)

## Magentic-One with Assistant Agent

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer


travel_agent = AssistantAgent(
    "TravelAgent",
    description="This agent provides a travel plan that includes travel destinations, flights, restaurants, and accommodations.",
    model_client=az_model_client,
    system_message="""
        You are a travel agent that will provide travel plans that in include travel destinations, flights, restaurants, and accommodations.
    """,
)

team = MagenticOneGroupChat([travel_agent], model_client=az_model_client)

task = """
    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.
"""
response = await Console(team.run_stream(task=task))



In [ ]:
get_agent_responses(response)

## Magentic-One with a MultimodalWebSurfer

In [ ]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=az_model_client,
    )

team = MagenticOneGroupChat([surfer], model_client=az_model_client)
response = await Console(team.run_stream(task="What was the latest trade news on the Los Angeles Lakers basketball team?"))

In [ ]:
get_agent_responses(response)

## Magentic-One with a Code Executor Agent

In [5]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

# Create a code executor agent that uses a Docker container to execute code.
code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
await code_executor.start()
code_executor_agent = CodeExecutorAgent("code_executor", code_executor=code_executor)

team = MagenticOneGroupChat([code_executor_agent], model_client=az_model_client, max_turns=50)
task = f"""
You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart sorted by descending order with the sightings in the Y-axis and years in the X-axis and save it to a PNG file.
Put the value on top of each bar

"""
response = await Console(team.run_stream(task=task))

# Stop the code executor.
await code_executor.stop()

---------- user ----------

You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart sorted by descending order with the sightings in the Y-axis and years in the X-axis and save it to a PNG file.
Put the value on top of each bar


---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart sorted by descending order with the sightings in the Y-axis and years in the X-axis and save it to a PNG file.
Put the value on top of each bar




To answer this request we have assembled the following team:

code_executor: A computer terminal that performs no other action th

In [8]:
get_agent_responses(response)


****************Agent Responses****************
user

You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart sorted by descending order with the sightings in the Y-axis and years in the X-axis and save it to a PNG file.
Put the value on top of each bar


MagenticOneOrchestrator

We are working to address the following user request:


You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart sorted by descending order with the sightings in the Y-axis and years in the X-axis and save it to a PNG file.
Put the value on top of each bar




To answer this request we have assembled the following team:

code_executor: A computer terminal that performs no other actio

## Magentic-One with File Surfer

In [9]:
from autogen_ext.agents.file_surfer._file_surfer import FileSurfer
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console

file_path = "Data/nasabooks/"
# Create a file surfer agent.
file_surfer = FileSurfer(
    "FileSurfer",
    model_client=az_model_client
)
file_surfer._browser.open_path(file_path)


team = MagenticOneGroupChat(
    [file_surfer],
    model_client=az_model_client, 
    max_turns=50,)

task = """What can you see in Mauritania?"""
response = await Console(team.run_stream(task=task))


/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:412: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["vision"] is False:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:427: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["json_output"] is False and json_output is True:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:433: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["function_calling"] is False and len(tools) > 0:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:149: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = a

---------- user ----------
What can you see in Mauritania?
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

What can you see in Mauritania?


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
- The request concerns the country of Mauritania.

2. FACTS TO LOOK UP
- Specific tourist attractions or natural landmarks in Mauritania.
- Historical or cultural sites in Mauritania.
- Major cities and places of interest in Mauritania.
- National parks or protected areas in Mauritania.
- Details about the geography and climate of Mauritania.

3. FACTS TO DERIVE
- None for this request, as the information needed is straightforward and should be covered by either looking up or recalling from memory.

4. EDUCATED GUESSES
- Mauritania is likely to have notable desert landscapes, such as parts of the Sahara Desert.
-

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:284: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, json_output=True)


---------- FileSurfer ----------
Path: /workspaces/azurehackathon/Data/nasabooks
Viewport position: Showing page 1 of 1.

# Index of /workspaces/azurehackathon/Data/nasabooks

| Name | Size | Date Modified |
| ---- | ---- | ------------- |
| .. (parent directory) | | |
| page-69.pdf | 317207 | 2025-02-03 08:32 |
| page-41.pdf | 317802 | 2025-02-03 08:32 |
| page-13.pdf | 318202 | 2025-02-03 08:32 |
| page-33.pdf | 317428 | 2025-02-03 08:32 |
| page-59.pdf | 316974 | 2025-02-03 08:32 |
| page-9.pdf | 177955 | 2025-02-03 08:32 |
| page-17.pdf | 318109 | 2025-02-03 08:32 |
| page-23.pdf | 317875 | 2025-02-03 08:32 |
| page-61.pdf | 317174 | 2025-02-03 08:32 |
| page-11.pdf | 318056 | 2025-02-03 08:32 |
| page-55.pdf | 316783 | 2025-02-03 08:32 |
| page-35.pdf | 318285 | 2025-02-03 08:32 |
| page-25.pdf | 317964 | 2025-02-03 08:32 |
| page-57.pdf | 317067 | 2025-02-03 08:32 |
| page-45.pdf | 317489 | 2025-02-03 08:32 |
| page-19.pdf | 318378 | 2025-02-03 08:32 |
| page-43.pdf | 318094 | 20

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/agents/file_surfer/_file_surfer.py:123: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  create_result = await self._model_client.create(


---------- MagenticOneOrchestrator ----------
Please research online and gather information about major cities and key points of interest in Mauritania, focusing on Nouakchott and Nouadhibou, as well as important landmarks and cultural sites.
---------- FileSurfer ----------
### Research Findings on Major Cities and Points of Interest in Mauritania: Nouakchott and Nouadhibou

#### 1. Nouakchott
**Overview**: Nouakchott is the capital and largest city of Mauritania. It is the political, economic, and cultural center of the country.

**Key Points of Interest**:
- **Port de Pêche**: This is the city's bustling fishing port where visitors can see colorful traditional pirogues and local fishermen in action. It's a great spot for photography and experiencing local life.
- **Nouakchott Beach**: Known for its beautiful coastline along the Atlantic Ocean. It's a popular place for relaxation and getting a fresh sea breeze.
- **National Museum of Mauritania**: Offers insights into the history, cu

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:400: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)


In [10]:
get_agent_responses(response)

****************Agent Responses****************
user
What can you see in Mauritania?
MagenticOneOrchestrator

We are working to address the following user request:

What can you see in Mauritania?


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
- The request concerns the country of Mauritania.

2. FACTS TO LOOK UP
- Specific tourist attractions or natural landmarks in Mauritania.
- Historical or cultural sites in Mauritania.
- Major cities and places of interest in Mauritania.
- National parks or protected areas in Mauritania.
- Details about the geography and climate of Mauritania.

3. FACTS TO DERIVE
- None for this request, as the information needed is straightforward and should be covered by either looking up or recalling from memory.

4. EDUCATED GUESSES
- Mauritania is likely to have notable desert landscapes, such as parts of the Sahara Deser